In [8]:
import pandas as pd
from river import naive_bayes, tree, metrics, drift
from sklearn.model_selection import train_test_split

In [9]:
def get_data():
    df = pd.read_csv("rt_8873985678962563_abrupto.csv")
    return df[['X1', 'X2']], df['class']

In [10]:
# Load the dataset
X, y = get_data()

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
# Initialize two base learners
nb_model = naive_bayes.GaussianNB()
ht_model = tree.HoeffdingTreeClassifier()

In [13]:
# Initialize ADWIN drift detector
adwin = drift.ADWIN()

In [14]:
# Initialize metrics
nb_metrics = {'accuracy': metrics.Accuracy(), 'auc': metrics.ROCAUC()}
ht_metrics = {'accuracy': metrics.Accuracy(), 'auc': metrics.ROCAUC()}

In [15]:
# Train and evaluate the models
for x, y_true in zip(X_train.to_dict(orient='records'), y_train):
    # Update ADWIN and check for drift
    adwin.update(y_true)
    if adwin.drift_detected:
        # If drift is detected, reinitialize the models
        nb_model = naive_bayes.GaussianNB()
        ht_model = tree.HoeffdingTreeClassifier()

    # Train the models
    nb_model.learn_one(x, y_true)
    ht_model.learn_one(x, y_true)

    # Update metrics
    for metric in nb_metrics.values():
        metric.update(y_true, nb_model.predict_one(x))
    for metric in ht_metrics.values():
        metric.update(y_true, ht_model.predict_one(x))

In [16]:
# Test the models
for x, y_true in zip(X_test.to_dict(orient='records'), y_test):
    for metric in nb_metrics.values():
        metric.update(y_true, nb_model.predict_one(x))
    for metric in ht_metrics.values():
        metric.update(y_true, ht_model.predict_one(x))

In [17]:
# Print the performance
print("Naive Bayes Performance:")
for name, metric in nb_metrics.items():
    print(f"{name}: {metric.get()}")

print("\nHoeffding Tree Performance:")
for name, metric in ht_metrics.items():
    print(f"{name}: {metric.get()}")


Naive Bayes Performance:
accuracy: 0.656725
auc: 0.654661007428267

Hoeffding Tree Performance:
accuracy: 0.67975
auc: 0.6786962489427549
